# Import libraries

In [173]:
# uncomment to install libraries
# ! pip install numpy pandas matplotlib seaborn requests selenium bs4 regex glob2 geopy geopandas
# ! pip install jupyter_contrib_nbextensions && jupyter contrib nbextension install 

import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import selenium
from bs4 import BeautifulSoup
import re
import time
import geopy
import geopandas

# Test web scraping escape room reviews

## World of Escapes map page URL

In [2]:
map_page_url = "https://worldofescapes.com/map"

# also home page
home_page_url = "https://worldofescapes.com"

## Map page response

In [3]:
map_page_response = requests.get(map_page_url)

## Map page soup

In [4]:
map_page_soup = BeautifulSoup(map_page_response.text, "html.parser")

## Room URLs within cities within states

In [110]:
# dictionary containers for room urls within cities within states
state_city_room_urls = {}

# loop through rooms within cities within states
for state_item_i in map_page_soup.find_all("div", {"class": "col-lg-3 col-md-4 col-sm-6 col-xs-6 state-item"}):
    # state string
    state_i = state_item_i.find("h3").get_text().strip()
    
    # add state-level dictionary
    state_city_room_urls[state_i] = {}
    
    # add cities to state values
    for city_item_j in state_item_i.find_all("li", {"class": "city-item"}):
        # city string
        city_j = city_item_j.find("a").get_text()
        
        # add city-level dictionary to the state-level dictionary
        state_city_room_urls[state_i][city_j] = {}
        
        # city url
        city_url_j = home_page_url + city_item_j.find("a")["href"]
        
        # city url response
        city_response_j = requests.get(city_url_j)
        
        # city url soup
        city_soup_j = BeautifulSoup(city_response_j.text, "html.parser")
        
        # sleep for between 1-20 seconds
        sleep_time_j = np.random.randint(low = 1, high = 20, size = 1)
        
        # print stage in loop
        print("Made soup for the {}, {} page. Sleeping for {} seconds.".format(city_j, state_i, str(sleep_time_j).strip("[]")))
        
        # now sleep
        time.sleep(sleep_time_j)
        
        # list containers for escape room company name, escape room game name, and url
        company_name = []
        room_name = []
        room_url = []
                
        # company title
        for company_link in city_soup_j.find_all("a", {"class": "company-link"}):
            company_name.append(str(re.findall('title="(.*?)"', str(company_link))).strip("['']"))
                
        # game title
        for room_link in city_soup_j.find_all("a", {"class": "quest_tile_name_link"}):
            room_name.append(room_link.get_text())
                    
        # concatenate company name: room name
        company_and_room = [": ".join([company, room]) for company, room in zip(company_name, room_name)]
                
        # add lists for company-room labels
        for company_and_room_k in company_and_room:
            state_city_room_urls[state_i][city_j][company_and_room_k] = []
                
        # loop through href for each room, append to home page to make url
        if city_soup_j.find("section", {"class": "container"}) is not None:
            for a in city_soup_j.find("section", {"class": "container"}).find_all("a", {"class": "item-hover quest_tile_hover_link"}):
                room_url.append(home_page_url + a["href"])
                
            # append room urls to room dictionaries
            for company_and_room_k, room_url_k in zip(company_and_room, room_url):
                state_city_room_urls[state_i][city_j][company_and_room_k].append(room_url_k)
                
                # print loop result
                print("Saved URL for {} in {}, {}.".format(company_and_room_k, city_j, state_i))
        else:
            print(None)

Made soup for the Auburn, Alabama page. Sleeping for 3 seconds.
Saved URL for Auburn Escape Zones: Imprisoned in Auburn, Alabama.
Saved URL for Auburn Escape Zones: The Cabin in Auburn, Alabama.
Saved URL for Auburn Escape Zones: Vault in Auburn, Alabama.
Saved URL for Auburn Escape Zones: The Puzzler vs Superheroes in Auburn, Alabama.
Saved URL for Auburn Escape Zones: Black Beard's Brig in Auburn, Alabama.
Saved URL for Auburn Escape Zones: Vault in Auburn, Alabama.
Saved URL for Auburn Escape Zones: The Puzzler vs Superheroes in Auburn, Alabama.
Saved URL for Auburn Escape Zones: The Cabin in Auburn, Alabama.
Saved URL for Auburn Escape Zones: Black Beard's Brig in Auburn, Alabama.
Saved URL for Auburn Escape Zones: Imprisoned in Auburn, Alabama.
Made soup for the Birmingham, Alabama page. Sleeping for 18 seconds.
Saved URL for Breakout Games: The Kidnapping in Birmingham, Alabama.
Saved URL for Breakout Games: Runaway Train in Birmingham, Alabama.
Saved URL for Breakout Games: Muse

Made soup for the Orange Beach, Alabama page. Sleeping for 8 seconds.
Saved URL for Breakout Games: Hostage in Orange Beach, Alabama.
Saved URL for Breakout Games: Operation: Casino in Orange Beach, Alabama.
Saved URL for Breakout Games: The Kidnapping in Orange Beach, Alabama.
Saved URL for Breakout Games: Island Escape in Orange Beach, Alabama.
Saved URL for Breakout Games: Runaway Train in Orange Beach, Alabama.
Saved URL for Gulf Coast Escape Room:  Bates Motel in Orange Beach, Alabama.
Saved URL for XIT Escape Room: CSI: Gulf Shores in Orange Beach, Alabama.
Saved URL for XIT Escape Room: Movie Night in Orange Beach, Alabama.
Saved URL for Breakout Games: Runaway Train in Orange Beach, Alabama.
Saved URL for Breakout Games: Island Escape in Orange Beach, Alabama.
Saved URL for Escape House Waterville: The Hurricane Room in Orange Beach, Alabama.
Saved URL for Get a Clue: Crime Scene in Orange Beach, Alabama.
Saved URL for Gulf Coast Escape Room: The Doll Room in Orange Beach, Alab

Made soup for the Prescott, Arizona page. Sleeping for 10 seconds.
Saved URL for Prescott Escape Room: Dracula Experience in Prescott, Arizona.
Saved URL for Prescott Escape Room: Dracula Experience in Prescott, Arizona.
Made soup for the Tucson, Arizona page. Sleeping for 1 seconds.
Saved URL for Escape Room Tucson: Area 53: Lunar Lockdown in Tucson, Arizona.
Saved URL for Escape Room Tucson: Red Rum Cottage in Tucson, Arizona.
Saved URL for Down the Rabbit Hole: The Asylum in Tucson, Arizona.
Saved URL for Ace of Escape: International Thief in Tucson, Arizona.
Saved URL for Fox in a Box: Bank Robbery in Tucson, Arizona.
Saved URL for Will You Escape?: Serial Killer in Tucson, Arizona.
Saved URL for Down the Rabbit Hole: The Rabbit Hole: Welcome to 221B! in Tucson, Arizona.
Saved URL for Escape Room Tucson: Inside Job in Tucson, Arizona.
Saved URL for Ace of Escape: Bad Medicine in Tucson, Arizona.
Saved URL for Ace of Escape: Beat the Clock in Tucson, Arizona.
Saved URL for Escape Ro

Made soup for the Jonesboro, Arkansas page. Sleeping for 3 seconds.
Saved URL for Locked Away Escapes: The Russian Bomb Party in Jonesboro, Arkansas.
Saved URL for Locked Away Escapes: The Psych Ward in Jonesboro, Arkansas.
Saved URL for Locked Away Escapes: The Attic in Jonesboro, Arkansas.
Saved URL for Locked Away Escapes: The Fed in Jonesboro, Arkansas.
Saved URL for Locked Away Escapes: Lost in the Ozarks in Jonesboro, Arkansas.
Saved URL for Panic Haus: The Operator in Jonesboro, Arkansas.
Saved URL for Locked Away Escapes: Lost in the Ozarks in Jonesboro, Arkansas.
Saved URL for Panic Haus: The Operator in Jonesboro, Arkansas.
Saved URL for Locked Away Escapes: The Fed in Jonesboro, Arkansas.
Saved URL for Locked Away Escapes: The Attic in Jonesboro, Arkansas.
Saved URL for Locked Away Escapes: The Psych Ward in Jonesboro, Arkansas.
Saved URL for Locked Away Escapes: The Russian Bomb Party in Jonesboro, Arkansas.
Made soup for the Little Rock, Arkansas page. Sleeping for 7 secon

Made soup for the Carlsbad, California page. Sleeping for 13 seconds.
Made soup for the Chico, California page. Sleeping for 4 seconds.
Saved URL for Chico Escape Rooms: Baker Street Mystery in Chico, California.
Saved URL for Chico Escape Rooms: Aunt Edna's Condo in Chico, California.
Saved URL for Chico Escape Rooms: The List in Chico, California.
Saved URL for Chico Escape Rooms: The List in Chico, California.
Saved URL for Chico Escape Rooms: Baker Street Mystery in Chico, California.
Saved URL for Chico Escape Rooms: Aunt Edna's Condo in Chico, California.
Made soup for the Eureka, California page. Sleeping for 19 seconds.
Saved URL for Escape Eureka: Da Vinci's Office in Eureka, California.
Saved URL for Escape Eureka: Quest for the Throne in Eureka, California.
Saved URL for Escape Eureka: Houdini's Final Act in Eureka, California.
Saved URL for Escape Eureka: Houdini's Final Act in Eureka, California.
Saved URL for Escape Eureka: Quest for the Throne in Eureka, California.
Save

...

Made soup for the Pensacola, Florida page. Sleeping for 8 seconds.
Saved URL for Escape on Palafox: Haunted in Pensacola, Florida.
Saved URL for Escape on Palafox: The Museum Robbery in Pensacola, Florida.
Saved URL for Escape LOL: Prison Escape in Pensacola, Florida.
Saved URL for Escape Zone 60: The Asylum in Pensacola, Florida.
Saved URL for Escape Zone 60: The Bank in Pensacola, Florida.
Saved URL for Exithis: Locked Up in Pensacola, Florida.
Saved URL for Exithis: Temple Ruins in Pensacola, Florida.
Saved URL for Escape on Palafox: It’s all WIRED! in Pensacola, Florida.
Saved URL for Exithis: Oval Office in Pensacola, Florida.
Saved URL for Cryptic Rooms: Escape the Maniac in Pensacola, Florida.
Saved URL for Cryptic Rooms: Magical Musical Mystery in Pensacola, Florida.
Saved URL for Escape LOL: Pirate Peril in Pensacola, Florida.
Saved URL for Escape on Palafox: It’s all WIRED! in Pensacola, Florida.
Saved URL for Exithis: Locked Up in Pensacola, Florida.
Saved URL for Exithis: O

Made soup for the West Palm Beach, Florida page. Sleeping for 1 seconds.
Saved URL for The Escape South Florida: Through The Looking Glass in West Palm Beach, Florida.
Saved URL for Room Raiders: Asylum in West Palm Beach, Florida.
Saved URL for Legends of Xscape: Ivan’s Lair (Nightmare Part 2) in West Palm Beach, Florida.
Saved URL for West Palm Beach Escape Rooms: The ClassRoom Adventure in West Palm Beach, Florida.
Saved URL for The Escape South Florida: Nightmare in The Gardens in West Palm Beach, Florida.
Saved URL for Time Travel Escapes: Cosmic Collision in West Palm Beach, Florida.
Saved URL for West Palm Beach Escape Rooms: Nuke M in West Palm Beach, Florida.
Saved URL for Room Raiders: Bloody Mary in West Palm Beach, Florida.
Saved URL for Extraordinary Escape Rooms: Swindled in West Palm Beach, Florida.
Saved URL for Legends of Xscape: The Elevator in West Palm Beach, Florida.
Saved URL for MindQuest Escape Games: Diamond Heist in West Palm Beach, Florida.
Saved URL for Time

Made soup for the Augusta, Georgia page. Sleeping for 6 seconds.
Saved URL for Source Code Escape Games: The Asylum in Augusta, Georgia.
Saved URL for Mastermind Escape Games: Outbreak: Find the Cure in Augusta, Georgia.
Saved URL for Mastermind Escape Games: Lost in Time in Augusta, Georgia.
Saved URL for Mastermind Escape Games: Bank Heist in Augusta, Georgia.
Saved URL for Mastermind Escape Games: Sorcerer's Secret in Augusta, Georgia.
Saved URL for Escapology: Mansion Murder in Augusta, Georgia.
Saved URL for Escapology: Th3 C0d3 in Augusta, Georgia.
Saved URL for Escapology: Cuban Crisis in Augusta, Georgia.
Saved URL for Source Code Escape Games: Hackers Headquarters in Augusta, Georgia.
Saved URL for Source Code Escape Games: The Da Vinci Exhibit in Augusta, Georgia.
Saved URL for Source Code Escape Games: The Da Vinci Exhibit in Augusta, Georgia.
Saved URL for Source Code Escape Games: Hackers Headquarters in Augusta, Georgia.
Saved URL for Escapology: Th3 C0d3 in Augusta, Geor

Made soup for the Kauai, Hawaii page. Sleeping for 5 seconds.
Saved URL for Kauai Escape Room: Curse of the Tiki Lounge in Kauai, Hawaii.
Saved URL for Kauai Escape Room: Captain Madok's Treasure in Kauai, Hawaii.
Saved URL for Kauai Escape Room: Quest For The Lost Continent in Kauai, Hawaii.
Saved URL for Kauai Escape Room: Captain Madok's Treasure in Kauai, Hawaii.
Made soup for the Maui, Hawaii page. Sleeping for 10 seconds.
Saved URL for Maui Escape Rooms: Prison Break in Maui, Hawaii.
Saved URL for Maui Escape Rooms: Ka Puka Bunker in Maui, Hawaii.
Saved URL for Maui Escape Rooms: Tesla's Secret Inheritance in Maui, Hawaii.
Saved URL for Maui Escape Rooms: Pirate Ship in Maui, Hawaii.
Saved URL for Mystery Maui: The Ramen Shop in Maui, Hawaii.
Saved URL for Mystery Maui: Stella Superstar in Maui, Hawaii.
Saved URL for Submerged Maui: Pele’s Stone Room in Maui, Hawaii.
Saved URL for Hawaii Escape Challenge: Maui Diamond Heist in Maui, Hawaii.
Saved URL for Hawaii Escape Challenge: 

ConnectionError: HTTPSConnectionPool(host='worldofescapes.com', port=443): Max retries exceeded with url: /coeur-d-alene (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f86339257d0>: Failed to establish a new connection: [Errno 50] Network is down'))

### Add Washington, D.C.

In [116]:
# add Washington, D.C. "state" container
state_city_room_urls["Washington, D.C."] = {}

# add Washington, D.C. "city" container
state_city_room_urls["Washington, D.C."]["Washington, D.C."] = {}

# soup
wdc_soup = BeautifulSoup(requests.get("https://worldofescapes.com/washington").text, "html.parser")

# list containers for escape room company name, escape room game name, and url
company_name = []
room_name = []
room_url = []
        
# company title
for company_link in wdc_soup.find_all("a", {"class": "company-link"}):
    company_name.append(str(re.findall('title="(.*?)"', str(company_link))).strip("['']"))
                
# game title
for room_link in wdc_soup.find_all("a", {"class": "quest_tile_name_link"}):
    room_name.append(room_link.get_text())
                    
# concatenate company name: room name
company_and_room = [": ".join([company, room]) for company, room in zip(company_name, room_name)]
                
# add lists for company-room labels
for company_and_room_k in company_and_room:
    state_city_room_urls["Washington, D.C."]["Washington, D.C."][company_and_room_k] = []
                
# loop through href for each room, append to home page to make url
for a in wdc_soup.find("section", {"class": "container"}).find_all("a", {"class": "item-hover quest_tile_hover_link"}):
    room_url.append(home_page_url + a["href"])
                
# append ...
for company_and_room_k, room_url_k in zip(company_and_room, room_url):
    state_city_room_urls["Washington, D.C."]["Washington, D.C."][company_and_room_k].append(room_url_k)
                
    # print loop result
    print("Saved URL for {} in {}, {}.".format(company_and_room_k, "Washington, D.C.", "Washington, D.C."))

# delete soup object
del wdc_soup

Saved URL for Breakout Games: Mystery Mansion in Washington, D.C., Washington, D.C..
Saved URL for Breakout Games: Hostage in Washington, D.C., Washington, D.C..
Saved URL for Breakout Games: Operation: Casino in Washington, D.C., Washington, D.C..
Saved URL for Breakout Games: The Kidnapping in Washington, D.C., Washington, D.C..
Saved URL for Breakout Games: Runaway Train in Washington, D.C., Washington, D.C..
Saved URL for Breakout Games: Island Escape in Washington, D.C., Washington, D.C..
Saved URL for Breakout Games: Undercover Alley in Washington, D.C., Washington, D.C..
Saved URL for Breakout Games: Undercover Alley in Washington, D.C., Washington, D.C..
Saved URL for Escape Room Live: Odditorium Emporium Shopping Spree in Washington, D.C., Washington, D.C..
Saved URL for Exit Plan: Refuge in Washington, D.C., Washington, D.C..
Saved URL for Room Escape DC: Magic School for Kids in Washington, D.C., Washington, D.C..
Saved URL for Escapology: Antidote in Washington, D.C., Washi

### Print URLs

In [118]:
state_city_room_urls

{'Alabama': {'Auburn': {'Auburn Escape Zones: Imprisoned': ['https://worldofescapes.com/auburn/quests/auburn-escape-zones-imprisoned',
    'https://worldofescapes.com/auburn/quests/auburn-escape-zones-imprisoned'],
   'Auburn Escape Zones: The Cabin': ['https://worldofescapes.com/auburn/quests/auburn-escape-zones-the-cabin',
    'https://worldofescapes.com/auburn/quests/auburn-escape-zones-the-cabin'],
   'Auburn Escape Zones: Vault': ['https://worldofescapes.com/auburn/quests/auburn-escape-zones-vault',
    'https://worldofescapes.com/auburn/quests/auburn-escape-zones-vault'],
   'Auburn Escape Zones: The Puzzler vs Superheroes': ['https://worldofescapes.com/auburn/quests/auburn-escape-zones-the-puzzler-vs-superheroes',
    'https://worldofescapes.com/auburn/quests/auburn-escape-zones-the-puzzler-vs-superheroes'],
   "Auburn Escape Zones: Black Beard's Brig": ['https://worldofescapes.com/auburn/quests/auburn-escape-zones-black-beards-brig',
    'https://worldofescapes.com/auburn/quest

### Store room urls in a data frame

In [121]:
# data frame container
state_city_room_url_df = pd.DataFrame(columns = ["state", "city", "company_and_room", "woe_room_url"])

# loop through city values nested in state keys
for state_key, city_value in state_city_room_urls.items():
    # loop through room values nested within city keys
    for city_key, room_value in city_value.items():
        # loop through urls nested in room keys
        for room_key, url_value in room_value.items():
            # loop through urls (some duplicates on the page)
            for url_key, url in enumerate(url_value):
                # append to data frame (append does not occur in place, so need to assign appended data frame)
                state_city_room_url_df = state_city_room_url_df.append(pd.DataFrame({"state": state_key, "city": city_key, "company_and_room": room_key, "woe_room_url": url}, index = [0]), ignore_index = True)

# See it
state_city_room_url_df

,state,city,company_and_room,woe_room_url
0,Alabama,Auburn,Auburn Escape Zones: Imprisoned,https://worldofescapes.com/auburn/quests/aubur...
1,Alabama,Auburn,Auburn Escape Zones: Imprisoned,https://worldofescapes.com/auburn/quests/aubur...
2,Alabama,Auburn,Auburn Escape Zones: The Cabin,https://worldofescapes.com/auburn/quests/aubur...
3,Alabama,Auburn,Auburn Escape Zones: The Cabin,https://worldofescapes.com/auburn/quests/aubur...
4,Alabama,Auburn,Auburn Escape Zones: Vault,https://worldofescapes.com/auburn/quests/aubur...
...,...,...,...,...
3153,"Washington, D.C.","Washington, D.C.",Escape Quest: The King’s Ransom,https://worldofescapes.com/washington/quests/e...
3154,"Washington, D.C.","Washington, D.C.",PanIQ Escape Room: Primal Quest,https://worldofescapes.com/washington/quests/p...
3155,"Washington, D.C.","Washington, D.C.",Escape Room Live: Ghostbusters!,https://worldofescapes.com/washington/quests/e...
3156,"Washington, D.C.","Washington, D.C.",Escape Room Live: Friday the 13th,https://worldofescapes.com/washington/quests/e...


### Drop duplicates and write .csv

In [123]:
# drop duplicatesa
state_city_room_url_df = state_city_room_url_df.drop_duplicates(inplace = False)

# write to .csv
state_city_room_url_df.to_csv("data/state_city_room_url_df.csv", index = False)

## Example page soup: Weird Heritage by Room 5280 in Denver, CO

In [6]:
room_soup_example = BeautifulSoup(requests.get("https://worldofescapes.com/denver/quests/room-5280-weird-heritage").text)

## Define functions for extracting information from room soup

### Room/Game Title

In [136]:
def get_title(soup):
    if soup.find("title") is not None:
        return soup.find("title").get_text()
    else:
        return "None"

# test
get_title(room_soup_example)

'Escape room "Weird Heritage" by ROOM 5280 in Denver'

### Room/Game Description

In [137]:
def get_description(soup):
    if soup.find("div", {"class": "description"}) is not None:
        return re.sub("Description:", "", soup.find("div", {"class": "description"}).get_text())
    else:
        return "None"

# test
get_description(room_soup_example)

'This is for real! You will find yourself in a strange apartment which you were lucky to inherit from a mysterious relative of yours. How cool is that? The only problem is that you have never heard of this “relative” before and what’s more important, now that the door is locked, you do not know how to escape!'

### Room/Game Address

In [138]:
def get_address(soup):
    if soup.find("div", {"data-content": "address"}) is not None:
        return re.sub(" \(Show on map\)", "", soup.find("div", {"data-content": "address"}).get_text().strip())
    else:
        return "None"

# test
get_address(room_soup_example)

'142 W 5th Ave, Denver, CO 80204'

### Room/Game Tags

In [139]:
def get_tags(soup):
    if soup.find("div", {"class": "tags"}) is not None:
        return [li.get_text() for li in soup.find("div", {"class": "tags"}).find("ul", {"class": "tags-2"}).find_all("li")]
    else:
        return ["None"]

# test
get_tags(room_soup_example)

['Public Ticketing', 'Up to ten players']

### Room/Game User Reviews

In [140]:
def get_reviews(soup):
    if soup.find("ul", {"class": "masonry-list"}) is not None:
        return [content.get_text() for content in soup.find("ul", {"class": "masonry-list"}).find_all("p", {"class": "content"})]
    else:
        return ["None"]

# test
get_reviews(room_soup_example)

['This place was awesome! For many in our group, this was our first escape room and it was the perfect amount of challenge. The room we did (Weird Heritage) was extremely well thought out and the puzzles were fun and interactive. The host was funny and gave us a hint only when we really needed it.',
 'We had a GREAT time in the Heritage room! There were six of us, including my nine-year-old daughter, and really I think you need a bare minimum of four to make this work, but the owner said you really need six. The owner is very hospitable and the clues are challenging but not impossible.We escaped with eight minutes to spare and were pretty happy about it!  Highly recommend.',
 'We did "Weird Heritage," and it was great! They had a good variety of puzzles and challenges. Can\'t wait to try another escape room here!',
 'Had a great time! Escaped The Heritage room. Greet for a larger group! Thanks so much!']

### Room/Game Player Range

In [141]:
def get_player_range(soup):
    if soup.find("ul", {"class": "params-ul"}).find("li", {"data-content": "participants-count"}) is not None:
        return re.sub("\u200a–\u200a", "-", soup.find("ul", {"class": "params-ul"}).find("li", {"data-content": "participants-count"}).get_text().strip("Number of players"))
    else:
        return "None"

# test
get_player_range(room_soup_example)

'4-10'

### Room/Game Time Limit

In [142]:
def get_time_limit(soup):
    if soup.find("ul", {"class": "params-ul"}).find("li", {"data-content": "time"}) is not None:
        return int(soup.find("ul", {"class": "params-ul"}).find("li", {"data-content": "time"}).get_text().split()[1].split("limit")[1])
    else:
        return "None"

# test
get_time_limit(room_soup_example)

60

### Room/Game Difficulty Level

In [143]:
def get_difficulty_level(soup):
    if soup.find("ul", {"class": "params-ul"}).find("span", {"data-toggle": "tooltip"}) is not None:
        return str(re.findall('data-original-title="(.*?)"', str(soup.find("ul", {"class": "params-ul"}).find("span", {"data-toggle": "tooltip"})))).strip("['']")
    else:
        return "None"

# test
get_difficulty_level(room_soup_example)

'Difficult'

### Room/Game Fear Level

In [144]:
def get_fear_level(soup):
    if soup.find("ul", {"class": "params-ul"}).find("span", {"class": "in-words"}) is not None:
        return soup.find("ul", {"class": "params-ul"}).find("span", {"class": "in-words"}).get_text()
    else:
        return "None"

# test
get_fear_level(room_soup_example)

'Not scary'

### Room/Game Age Requirement

In [145]:
def get_age_requirement(soup):
    if soup.select(".cell:nth-child(5) .td") is not None:
        return str(re.findall('data-toggle="tooltip">(.*?)<', str(soup.select(".cell:nth-child(5) .td")))).strip("['']")
    else:
        return "None"

# test
get_age_requirement(room_soup_example)

'12+'

### Room/Game Success Rate

In [146]:
def get_success_rate(soup):
    if soup.find("span", {"data-original-title": "Success rate:"}) is not None:
        return float(soup.find("span", {"data-original-title": "Success rate:"}).get_text().strip(" %")) / 100
    else:
        return "None"

# test
get_success_rate(room_soup_example)

0.3

## Combine functions into one

### Most room data except reviews and tags

In [147]:
def get_room_data(url, soup):
    return pd.DataFrame({"room_title": get_title(soup), "woe_room_url": url, "room_address": get_address(soup), "room_description": get_description(soup), "player_range": get_player_range(soup), "time_limit": get_time_limit(soup), "difficulty_level": get_difficulty_level(soup), "fear_level": get_fear_level(soup), "age_requirement": get_age_requirement(soup), "success_rate": get_success_rate(soup)}, index = [0])

# test
get_room_data("https://worldofescapes.com/denver/quests/room-5280-weird-heritage", room_soup_example)

,room_title,woe_room_url,room_address,room_description,player_range,time_limit,difficulty_level,fear_level,age_requirement,success_rate
0,"Escape room ""Weird Heritage"" by ROOM 5280 in D...",https://worldofescapes.com/denver/quests/room-...,"142 W 5th Ave, Denver, CO 80204",This is for real! You will find yourself in a ...,4-10,60,Difficult,Not scary,12+,0.3


### Review data

In [148]:
def get_review_data(url, soup):
    # room title and reviews
    room_title = get_title(soup)
    reviews = get_reviews(soup)
    
    # data frame container
    data_frame = pd.DataFrame(columns = ["room_title", "woe_room_url", "review_id", "review"])
    
    # loop through reviews
    for i, review_i in enumerate(reviews):
        data_frame = data_frame.append(pd.DataFrame({"room_title": room_title, "woe_room_url": url, "review_id": i, "review": review_i}, index = [0]), ignore_index = True)
    
    return data_frame

# test
get_review_data("https://worldofescapes.com/denver/quests/room-5280-weird-heritage", room_soup_example)

,room_title,woe_room_url,review_id,review
0,"Escape room ""Weird Heritage"" by ROOM 5280 in D...",https://worldofescapes.com/denver/quests/room-...,0,"This place was awesome! For many in our group,..."
1,"Escape room ""Weird Heritage"" by ROOM 5280 in D...",https://worldofescapes.com/denver/quests/room-...,1,We had a GREAT time in the Heritage room! Ther...
2,"Escape room ""Weird Heritage"" by ROOM 5280 in D...",https://worldofescapes.com/denver/quests/room-...,2,"We did ""Weird Heritage,"" and it was great! The..."
3,"Escape room ""Weird Heritage"" by ROOM 5280 in D...",https://worldofescapes.com/denver/quests/room-...,3,Had a great time! Escaped The Heritage room. G...


### Tags data

In [149]:
def get_tag_data(url, soup):
    # room title and reviews
    room_title = get_title(soup)
    tags = get_tags(soup)
    
    # data frame container
    data_frame = pd.DataFrame(columns = ["room_title", "woe_room_url", "tag_id", "tag"])
    
    # loop through reviews
    for i, tag_i in enumerate(tags):
        data_frame = data_frame.append(pd.DataFrame({"room_title": room_title, "woe_room_url": url, "tag_id": i, "tag": tag_i}, index = [0]), ignore_index = True)
    
    return data_frame

# test
get_tag_data("https://worldofescapes.com/denver/quests/room-5280-weird-heritage", room_soup_example)

,room_title,woe_room_url,tag_id,tag
0,"Escape room ""Weird Heritage"" by ROOM 5280 in D...",https://worldofescapes.com/denver/quests/room-...,0,Public Ticketing
1,"Escape room ""Weird Heritage"" by ROOM 5280 in D...",https://worldofescapes.com/denver/quests/room-...,1,Up to ten players


## Loop through room URLs, extract data, and write to .csv files

In [162]:
for url in state_city_room_url_df["woe_room_url"][1000:2001]:
    # href for naming .csv
    room_href = url.split("quests/")[1]
    
    # soup
    soup = BeautifulSoup(requests.get(url).text, "html.parser")
    
    # write room data to .csv
    get_room_data(url, soup).to_csv("data/room_data/" + room_href + "_room_data.csv", index = False)
    
    # write review data to .csv
    get_review_data(url, soup).to_csv("data/reviews/" + room_href + "_room_reviews.csv", index = False)
    
    # write tag data to .csv
    get_tag_data(url, soup).to_csv("data/tags/" + room_href + "_room_tags.csv", index = False)
        
    # sleep for between 1-20 seconds
    sleep_time_i = np.random.randint(low = 1, high = 20, size = 1)
    
    # print iteration results
    print("Exported .csv files for {}. Waiting {} seconds before making more soup.".format(room_href, str(sleep_time_i).strip("[]")))
    
    # now sleep
    time.sleep(sleep_time_i)

Exported .csv files for real-escape-room-popstars-room-of-doom. Waiting 17 seconds before making more soup.
Exported .csv files for clockwise-escape-room-the-paranormal-experiment. Waiting 18 seconds before making more soup.
Exported .csv files for clues-and-gumshoes-captivated-at-sea. Waiting 8 seconds before making more soup.
Exported .csv files for pacifica-escape-zone-house-in-the-woods. Waiting 3 seconds before making more soup.
Exported .csv files for that-escape-room-a-night-in-the-fashion-store. Waiting 5 seconds before making more soup.
Exported .csv files for escape-oakland-spy-game. Waiting 8 seconds before making more soup.
Exported .csv files for sandbox-vr-ufl. Waiting 17 seconds before making more soup.
Exported .csv files for clues-and-gumshoes-the-show-must-go-on. Waiting 15 seconds before making more soup.
Exported .csv files for sandbox-vr-star-trek-discovery. Waiting 13 seconds before making more soup.
Exported .csv files for room-omega-ca. Waiting 11 seconds before

Exported .csv files for kuma-escape-the-strange-room. Waiting 14 seconds before making more soup.
Exported .csv files for paradox-escape-rooms-independence. Waiting 10 seconds before making more soup.
Exported .csv files for kuma-escape-social-ranking. Waiting 5 seconds before making more soup.
Exported .csv files for quadra-escape-remote-chance-a-cinematic-escape. Waiting 3 seconds before making more soup.
Exported .csv files for mighty-awesome-escape-rooms-the-office. Waiting 6 seconds before making more soup.
Exported .csv files for mighty-awesome-escape-rooms-terror-in-tinseltown. Waiting 7 seconds before making more soup.
Exported .csv files for locked-adventures-quest-for-the-throne. Waiting 14 seconds before making more soup.
Exported .csv files for locked-adventures-dystopia. Waiting 17 seconds before making more soup.
Exported .csv files for clues-and-wits-escape-rooms-sassafras-saloon. Waiting 1 seconds before making more soup.
Exported .csv files for locked-adventures-the-cu

AttributeError: 'NoneType' object has no attribute 'find'

### Read room data .csv files and concatenate together

In [248]:
room_data = pd.concat([pd.read_csv(filepath_i) for filepath_i in glob.glob("data/room_data/*.csv")], ignore_index = True)

# See it
room_data

,room_title,woe_room_url,room_address,room_description,player_range,time_limit,difficulty_level,fear_level,age_requirement,success_rate
0,"Escape room ""The Lost Antidote"" by The Escape ...",https://worldofescapes.com/little-rock/quests/...,"1214 S Main St Little Rock, AR 72202","Dr. Stapleton, an evil scientist, has poisoned...",2-7,60,Average,Not scary,13+,0.4
1,"Escape room ""Tiki Time!"" by 60out Escape Rooms...",https://worldofescapes.com/los-angeles/quests/...,"2284 S Figueroa St Los Angeles, CA 90007",Welcome to the islands! You’ve traveled to the...,4-8,75,Very difficult,Not scary,12+,None
2,"Escape room ""Central Bank"" by Fox in a Box Ora...",https://worldofescapes.com/orange-county/quest...,"123 W. Amerige Ave., Fullerton CA 92832",You are a group of well-informed thieves. It h...,2-6,60,Difficult,Not scary,4+,0.23
3,"Escape room ""The Executioner"" by Huntsville Es...",https://worldofescapes.com/huntsville/quests/h...,"604 Madison St SE Huntsville, AL 35801","Long ago, in medieval times, fear has permeate...",2-7,60,Average,Not scary,8+,None
4,"Escape room ""World of Illusions"" by Maze Rooms...",https://worldofescapes.com/los-angeles/quests/...,"1328 N Highland Ave, Los Angeles, CA 90028",Welcome to the world of illusions! Learn the G...,2-8,60,Difficult,Not scary,NaN,None
...,...,...,...,...,...,...,...,...,...,...
1097,"Escape room ""The Dragon's Lair"" by Maze Rooms ...",https://worldofescapes.com/los-angeles/quests/...,"4365 Sepulveda Blvd, Culver City, CA 90230",Unravel the mystery to the disappearance of yo...,2-7,60,Difficult,Not scary,12+,None
1098,"Escape room ""Double Agent General"" by Great Es...",https://worldofescapes.com/phoenix/quests/grea...,44301 W Maricopa-Casa Grande Hwy #104 Maricopa...,General Matthew Ridgeway has been working as a...,2-6,60,Difficult,Not scary,16+,0.6
1099,"Escape room ""Sister's Secret"" by The Unlockabl...",https://worldofescapes.com/san-diego/quests/th...,"7380 Clairemont Mesa Blvd Suite 205 San Diego,...",As part of the Paranormal Division of the Bure...,3-6,60,Difficult,None,16+,None
1100,"Escape room ""Hollywood Dreams"" by Escape Room ...",https://worldofescapes.com/los-angeles/quests/...,3605 Long Beach Boulevard Suite 304 Long Beach...,Hollywood is hard enough to break into for asp...,4-6,60,Average,Not scary,14+,None


#### Recode room variables

In [249]:
# recode difficulty_level
def recode_difficulty(x):
    if x == "Easy":
        return 0
    elif x == "Average":
        return 1
    elif x == "Difficult":
        return 2
    elif x == "Very difficult":
        return 3
    else:
        return "None"

# recode fear_level
def recode_fear(x):
    if x == "Not scary":
        return 0
    elif x == "Scary":
        return 1
    elif x == "Very scary":
        return 2
    else:
        return "None"

# modify variables
room_data = room_data.assign(min_players = room_data["player_range"].apply(lambda x: x[:1]),
                             max_players = room_data["player_range"].apply(lambda x: x.split("-")[1]),
                             min_age = room_data["age_requirement"].apply(lambda x: str(x).strip("+")),
                             difficulty_int = room_data["difficulty_level"].apply(lambda x: recode_difficulty(x)),
                             fear_int = room_data["fear_level"].apply(lambda x: recode_fear(x)))

#### Add latitude and longitude

In [244]:
# Geocoder using the Google Maps v3 API
googlev3_locator = geopy.geocoders.GoogleV3(api_key = "KEY_GOES_HERE")
    
# address, latitude, and longitude
def get_lat_long(address):
    # look up location
    location = googlev3_locator.geocode(address)
    
    return location.address, location.latitude, location.longitude

# query address, latitude, and longitude columns
query_address, latitude, longitude = [], [], []
for address in room_data["room_address"]:
    # get
    address_lat_long = get_lat_long(address)
    
    # append
    query_address.append(address_lat_long[0])
    latitude.append(address_lat_long[1])
    longitude.append(address_lat_long[2])

TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"

In [253]:
room_data["query_address"] = query_address
room_data["room_latitude"] = latitude
room_data["room_longitude"] = longitude

# see it
room_data

,room_title,woe_room_url,room_address,room_description,player_range,time_limit,difficulty_level,fear_level,age_requirement,success_rate,min_players,max_players,min_age,difficulty_int,fear_int,query_address,room_latitude,room_longitude
0,"Escape room ""The Lost Antidote"" by The Escape ...",https://worldofescapes.com/little-rock/quests/...,"1214 S Main St Little Rock, AR 72202","Dr. Stapleton, an evil scientist, has poisoned...",2-7,60,Average,Not scary,13+,0.4,2,7,13,1,0,"1214 S Main St, Little Rock, AR 72202, USA",34.736756,-92.272776
1,"Escape room ""Tiki Time!"" by 60out Escape Rooms...",https://worldofescapes.com/los-angeles/quests/...,"2284 S Figueroa St Los Angeles, CA 90007",Welcome to the islands! You’ve traveled to the...,4-8,75,Very difficult,Not scary,12+,None,4,8,12,3,0,"2284 S Figueroa St, Los Angeles, CA 90007, USA",34.030983,-118.274007
2,"Escape room ""Central Bank"" by Fox in a Box Ora...",https://worldofescapes.com/orange-county/quest...,"123 W. Amerige Ave., Fullerton CA 92832",You are a group of well-informed thieves. It h...,2-6,60,Difficult,Not scary,4+,0.23,2,6,4,2,0,"123 W Amerige Ave, Fullerton, CA 92832, USA",33.872032,-117.925247
3,"Escape room ""The Executioner"" by Huntsville Es...",https://worldofescapes.com/huntsville/quests/h...,"604 Madison St SE Huntsville, AL 35801","Long ago, in medieval times, fear has permeate...",2-7,60,Average,Not scary,8+,None,2,7,8,1,0,"604 Madison St SE, Huntsville, AL 35801, USA",34.725170,-86.582573
4,"Escape room ""World of Illusions"" by Maze Rooms...",https://worldofescapes.com/los-angeles/quests/...,"1328 N Highland Ave, Los Angeles, CA 90028",Welcome to the world of illusions! Learn the G...,2-8,60,Difficult,Not scary,NaN,None,2,8,nan,2,0,"1328 N Highland Ave, Los Angeles, CA 90028, USA",34.095165,-118.338182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1097,"Escape room ""The Dragon's Lair"" by Maze Rooms ...",https://worldofescapes.com/los-angeles/quests/...,"4365 Sepulveda Blvd, Culver City, CA 90230",Unravel the mystery to the disappearance of yo...,2-7,60,Difficult,Not scary,12+,None,2,7,12,2,0,"4365 Sepulveda Blvd, Culver City, CA 90230, USA",34.003230,-118.407880
1098,"Escape room ""Double Agent General"" by Great Es...",https://worldofescapes.com/phoenix/quests/grea...,44301 W Maricopa-Casa Grande Hwy #104 Maricopa...,General Matthew Ridgeway has been working as a...,2-6,60,Difficult,Not scary,16+,0.6,2,6,16,2,0,"44301 W Maricopa-Casa Grande Hwy #104, Maricop...",33.054204,-112.043924
1099,"Escape room ""Sister's Secret"" by The Unlockabl...",https://worldofescapes.com/san-diego/quests/th...,"7380 Clairemont Mesa Blvd Suite 205 San Diego,...",As part of the Paranormal Division of the Bure...,3-6,60,Difficult,None,16+,None,3,6,16,2,None,"7380 Clairemont Mesa Blvd #205, San Diego, CA ...",32.832950,-117.159954
1100,"Escape room ""Hollywood Dreams"" by Escape Room ...",https://worldofescapes.com/los-angeles/quests/...,3605 Long Beach Boulevard Suite 304 Long Beach...,Hollywood is hard enough to break into for asp...,4-6,60,Average,Not scary,14+,None,4,6,14,1,0,"3605 Long Beach Blvd #304, Long Beach, CA 9080...",33.822674,-118.189654


### Read review data .csv files and concatenate together

In [254]:
review_data = pd.concat([pd.read_csv(filepath_i) for filepath_i in glob.glob("data/reviews/*.csv")], ignore_index = True)

# See it
review_data

,room_title,woe_room_url,review_id,review
0,"Escape room ""The Lost Antidote"" by The Escape ...",https://worldofescapes.com/little-rock/quests/...,0,None
1,"Escape room ""Last Will & Testament"" by House o...",https://worldofescapes.com/san-marcos-ca/quest...,0,Best room so far. You really feel as though y...
2,"Escape room ""221B Baker Street"" by Beat the Ro...",https://worldofescapes.com/sacramento/quests/b...,0,Took my boyfriend for our anniversary and this...
3,"Escape room ""221B Baker Street"" by Beat the Ro...",https://worldofescapes.com/sacramento/quests/b...,1,Did the Sherlock room and it was amazing. Fun ...
4,"Escape room ""221B Baker Street"" by Beat the Ro...",https://worldofescapes.com/sacramento/quests/b...,2,Although we didn't solve the mystery of who ha...
...,...,...,...,...
3391,"Escape room ""Ctrl + Alt + Reality"" by Planet E...",https://worldofescapes.com/los-angeles/quests/...,1,If you want a little challenge Try out CTRL+Al...
3392,"Escape room ""Ctrl + Alt + Reality"" by Planet E...",https://worldofescapes.com/los-angeles/quests/...,2,So fun! Incredibly detailed ambiance. Puzzles ...
3393,"Escape room ""It's Mine"" by Avalanche Escape Ro...",https://worldofescapes.com/anchorage/quests/av...,0,None
3394,"Escape room ""Chasm"" by Beat the Room in Sacram...",https://worldofescapes.com/sacramento/quests/b...,0,We had 6 family members working together and h...


### Read tag data .csv files and concatenate together

In [255]:
tag_data = pd.concat([pd.read_csv(filepath_i) for filepath_i in glob.glob("data/tags/*.csv")], ignore_index = True)

# See it
tag_data

,room_title,woe_room_url,tag_id,tag
0,"Escape room ""The Lab 51"" by Exit Game in Los A...",https://worldofescapes.com/los-angeles/quests/...,0,Beginners
1,"Escape room ""The Lab 51"" by Exit Game in Los A...",https://worldofescapes.com/los-angeles/quests/...,1,Futuristic
2,"Escape room ""The Lab 51"" by Exit Game in Los A...",https://worldofescapes.com/los-angeles/quests/...,2,Public Ticketing
3,"Escape room ""The Lab 51"" by Exit Game in Los A...",https://worldofescapes.com/los-angeles/quests/...,3,Up to ten players
4,"Escape room ""Human Xperiment"" by Xcape House i...",https://worldofescapes.com/tucson/quests/xscap...,0,Scary
...,...,...,...,...
3679,"Escape room ""Runaway Train"" by Breakout Orange...",https://worldofescapes.com/gulf-shores/quests/...,1,Private Ticketing
3680,"Escape room ""Runaway Train"" by Breakout Orange...",https://worldofescapes.com/gulf-shores/quests/...,2,Immersive
3681,"Escape room ""Runaway Train"" by Breakout Orange...",https://worldofescapes.com/gulf-shores/quests/...,3,Up to eight players
3682,"Escape room ""Masquerade Manor"" by Escape Room ...",https://worldofescapes.com/salinas/quests/esca...,0,Public Ticketing


## Export merged data frames to .csv files

In [256]:
room_data.to_csv("data/room_data.csv", index = False)
review_data.to_csv("data/review_data.csv", index = False)
tag_data.to_csv("data/tag_data.csv", index = False)